# Example hologram processing pipeline

### Step 00 - configure warnings and autoreload

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

### Step 01 - Imports

In [ ]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import os

from pyopia.classify import Classify
import pyopia.process
import pyopia.io
import pyopia.background
import pyopia.statistics
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

### Step 02 - (run once) Download test data set

In [ ]:
# download the holo test data
# either holo_test_data_01 (default) or holo_test_data_02
infolder = exampledata.get_folder_from_holo_repository("holo_test_data_01")

### Step 03 - Define runtime settings

In [ ]:
# path to folder of input data
infolder = 'holo_test_data_01'

# path to folder of output data (will be created)
outfolder = 'proc1'

# name of output statistics file (if already exists will be removed)
outfilename = 'holotest'

# path to classifier model to use (here we download and use an example model)
model_path = exampledata.get_example_model()

# segmentation threshold 
threshold = 0.9

# number of images to use as background (recommend >=10) - for either static or moving background removal
average_window = 10  

# hologram reconstruction settings
holo_initial_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'n': 1.33, # index of refraction of sample volume medium (1.33 for water)
                        'offset': 26, # offset to start of sample volume in mm
                        'minZ': 0, # minimum reconstruction distance within sample volume in mm
                        'maxZ': 50, # maximum reconstruction distance within sample volume in mm
                        'stepZ': 0.5} #step size in mm

### Step 04 - Setup folders & input file lists configured in Step 03

In [ ]:
# make output folder
os.makedirs(outfolder, exist_ok=True)

# remove pre-existing output file (as statistics for each image are appended to it)
datafile_hdf = os.path.join(outfolder,'holotest')
if os.path.exists(datafile_hdf + '-STATS.h5'):
  os.remove(datafile_hdf + '-STATS.h5')

# get sorted list of input files
files = sorted(glob(os.path.join(infolder, '*.pgm')))

# create list of files for initial creation of background file
bgfiles = files[:average_window]

### Step 05 - Setup pipeline steps

In [ ]:
# define the steps to use in the processing pipeline - given as a label and the Class with options
steps = {
         # initial step, run once on pipeline initialisation - arguments a filename to use for sizing the image and hologram reconstruction settings from Step 03
         'initial': holo.Initial(files[0], **holo_initial_settings), 
         # sets up classifier model, runs once on pipeline initialisation - argument is the path to the classification model to use from Step 03
         'classifier': Classify(model_path=model_path), 
         # creates initial background, runs once on pipeline initialisation - arguments are list of files to use for initial background and instrument-specific image loading function
         'create background': pyopia.background.CreateBackground(bgfiles, pyopia.instrument.holo.load_image),
         ### start of steps applied to every image
         # load the image using instrument-specific loading function 
         'load': holo.Load(),
         # apply background corrction - argument is which method to use:
         # pyopia.background.shift_bgstack_accurate - recommended method for moving background
         # pyopia.background.shift_bgstack_fast - faster method for realtime applications
         # omit argument for static background removal
         # change to pyopia.background.CorrectBackgroundNone() to not remove a background during pipeline
         'correct background': pyopia.background.CorrectBackgroundAccurate(pyopia.background.shift_bgstack_accurate),
         # hologram reconstruction step - argument is how much stack cleaning (% dimmest pixels to remove) to apply - set to 0 to omit cleaning
         'reconstruct': holo.Reconstruct(stack_clean=0.02),
         # focussing step - arguments are which stack summarisation method to use, what global segmentation threshold to apply and what focus method to use:
         # summarisation methods are:
         # pyopia.instrument.holo.std_map (default) - takes standard deviation of values through stack
         # pyopia.instrument.holo.max_map - takes maximum intensity value through stack
         # focus functions are:
         # pyopia.instrument.holo.find_focis_imax (default) - finds focus using plane of maximum intensity
         # pyopia.instrument.holo.find_focus_sobel - finds focus using edge sharpness
         # focus options are:
         # increase_depth_of_field (bool, default False) - finds max of planes adjacent to optimum focus plane
         # merge_adjacent_particles (int, default 0) - merges adjacent particles within stack summary using this pixel radius
         'focus': holo.Focus(pyopia.instrument.holo.std_map,threshold=threshold,focus_function=pyopia.instrument.holo.find_focus_sobel,increase_depth_of_field=True,merge_adjacent_particles=0),
         # segmentation of focussed particles - argument is threshold to apply (can be different to Focus step)
         'segmentation': pyopia.process.Segment(threshold=threshold),
         # extraction of particle statistics - arguments are:
         # export_outputpath - is output folder for image-specific outputs for montage creation (can be omitted)
         # additional_propnames - is list of additional skimage regionprops to export to stats (can be omitted)
         'statextract': pyopia.process.CalculateStats(export_outputpath=outfolder, additional_propnames=['feret_diameter_max', 'equivalent_diameter_area']),
         # step to merge hologram-specific information (currently focus depth & original filename) into output statistics file
         'merge holo stats': holo.MergeStats(),
         # write the output HDF5 statistics file
         'output': pyopia.io.StatsH5(datafile_hdf)
         }

# now initialise the pipeline
processing_pipeline = Pipeline(steps)

### Step 06 - Run the pipeline

In [ ]:
#loop through file list - or here just use the first 5 files
for filename in files[:5]:
    stats = processing_pipeline.run(filename)

### Step 07 - Review outputs

In [ ]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))

# Calculate the volume distribution from the stats DataFrame.
dias, vd = pyopia.statistics.vd_from_stats(stats, holo_initial_settings['pixel_size'])

# plot the volume distribution
plt.style.use('dark_background')
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

In [ ]:
# plot histogram of focus locations
import numpy as np
plt.style.use('dark_background')
zval = np.arange(holo_initial_settings['minZ'], holo_initial_settings['maxZ'], holo_initial_settings['stepZ'])
plt.hist(zval[stats.ifocus-1],len(zval))
plt.xlim(zval[0],zval[-1])
plt.xlabel('Z [mm]')

In [ ]:
# create montage of focussed particles
im_mont = pyopia.statistics.make_montage(datafile_hdf + '-STATS.h5',holo_initial_settings["pixel_size"],outfolder,
    auto_scaler=1000, msize=2048, maxlength=100000, crop_stats=None)
pyopia.statistics.montage_plot(im_mont,holo_initial_settings['pixel_size'])